# Optimization Example for Interim Operational Capability 1

## Set up.

### Import packages.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

## Load data.

### The data are stored in a set of tab-separated value files in a folder.

In [3]:
designs = ty.Designs("data")

In [4]:
investments = ty.Investments("data")

### Compile the production and metric functions for each technology in the dataset.

In [5]:
designs.compile()

## Evaluate the tranches of investments.

In [6]:
tranche_results = investments.evaluate_tranches(designs, sample_count=250)

## Create an evaluator to interpolate investments.

In [7]:
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

### Maximum investment amounts, by category.

In [8]:
evaluator.max_amount

,Amount
Category,
CIGS,3000000.0
CdTe,6000000.0
GaAs,7500000.0
InGaP,9500000.0
Perovskite,9500000.0
Polysilicon,4500000.0
Power Electronics,5000000.0
Soft Costs,5000000.0


### Range of metrics.

In [9]:
evaluator.min_metric.rename(columns = {"Value" : "Minimum"}).join(
    evaluator.max_metric.rename(columns = {"Value" : "Maximum"})
)

,Minimum,Maximum
Index,,
Capital,-3.870652,4.919771
Efficiency,1.880353,2.539163
GHG,-0.003592,-0.003592
Hazardous,0.706043,1.094999
LCOE,-0.260030,0.135016
Lifetime,185.231022,190.786049
Strategic,0.035692,0.076723
Yield,9766.933292,10651.491080


## ε-Constraint multiobjective optimization

### Create an optimizer.

In [10]:
optimizer = ty.EpsilonConstraintOptimizer(evaluator)

### Optimize on a single metric.

In [11]:
optimum = optimizer.maximize_slsqp("LCOE")
optimum.exit_message

/nix/store/2hqyq1p29z76wvh920r43a63sy9n1cag-python3-3.7.6-env/lib/python3.7/site-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/nix/store/2hqyq1p29z76wvh920r43a63sy9n1cag-python3-3.7.6-env/lib/python3.7/site-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


'Singular matrix E in LSQ subproblem'

Optimal investment amounts:

In [12]:
optimum.amounts

Category
CIGS                NaN
CdTe                NaN
GaAs                NaN
InGaP               NaN
Perovskite          NaN
Polysilicon         NaN
Power Electronics   NaN
Soft Costs          NaN
Name: Amount, dtype: float64

Values of metrics at optimal investment:

In [13]:
optimum.metrics

Index
Capital      NaN
Efficiency   NaN
GHG          NaN
Hazardous    NaN
LCOE         NaN
Lifetime     NaN
Strategic    NaN
Yield        NaN
Name: Value, dtype: float64